# Crawl điểm chuẩn 2024

In [1]:
# !pip install undetected-chromedriver selenium

In [1]:
import os
import time
import traceback
import pandas as pd
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException

In [ ]:
# os.chdir(r'D:\University\Applied Data Science\ADS_Final')

In [3]:
def close_ads_if_present(driver, wait, max_wait=2):
    # 1. Kiểm tra nếu bị redirect sang quảng cáo Google Vignette toàn trang
    try:
        if "#google_vignette" in driver.current_url:
            driver.back()
            wait.until(lambda d: "#google_vignette" not in d.current_url)
            print("[ADS] Đã tự động quay lại khi gặp quảng cáo Google Vignette.")
            time.sleep(1)
    except Exception as e:
        print("[ADS] Lỗi khi xử lý quảng cáo Google Vignette:", e)

    # 2. Kiểm tra và tắt modal quảng cáo nếu có (Ant Design popup)
    try:
        ad_close = WebDriverWait(driver, max_wait).until(
            EC.element_to_be_clickable((
                By.XPATH, "//div[contains(@class, 'ant-modal')]//button[contains(@class, 'ant-modal-close')]"
            ))
        )
        driver.execute_script("arguments[0].click();", ad_close)
        print("[ADS] ❌ Đã tự động tắt popup quảng cáo (ant-modal).")
        time.sleep(1)
    except:
        pass

## Phiên bản thành công cũ nhất (nhưng còn lỗi) =)))


In [7]:
# ========== 1. SETUP TRÌNH DUYỆT ==========
options = uc.ChromeOptions()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

profile_directory = "Profile"
os.makedirs(profile_directory, exist_ok=True)
options.user_data_dir = profile_directory

driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, 15)
action = ActionChains(driver)

# ========== 2. TRUY CẬP TRANG ==========
target_url = "https://diemthi.tuyensinh247.com/nganh-dao-tao.html#undefined"
driver.get(target_url)
close_ads_if_present(driver, wait)

try:
    see_more = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Xem thêm')]"))
    )
    driver.execute_script("arguments[0].click();", see_more)
    print("[DEBUG] Đã click 'Xem thêm' để hiển thị thêm nhóm ngành.")
    time.sleep(2)
except:
    print("[WARN] Không tìm thấy hoặc không thể click nút 'Xem thêm'.")

# ========== 3. THU THẬP DANH SÁCH NGÀNH ==========
group_data = []
printed_group_names = set()
seen_major_links = set()

try:
    print("[DEBUG] Đang chờ lấy danh sách nhóm ngành...")
    wait.until(lambda d: len(d.find_elements(By.XPATH, '//div[@role="tablist"]/div')) >= 30)
    groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
    print(f"[DEBUG] Tìm thấy {len(groups)} nhóm ngành.")

    for group_index in range(len(groups)):
        try:
            groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
            group = groups[group_index]
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", group)
            time.sleep(1)

            success = False
            for attempt in range(3):
                try:
                    groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
                    group = groups[group_index]
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", group)
                    time.sleep(1.2)

                    try:
                        action.move_to_element(group).click().perform()
                    except:
                        driver.execute_script("arguments[0].click();", group)

                    WebDriverWait(driver, 5).until(
                        lambda d: 'aria-expanded="true"' in group.get_attribute("outerHTML")
                    )

                    WebDriverWait(driver, 5).until(
                        lambda d: len(group.find_element(
                            By.XPATH, './/div[contains(@class, "ant-collapse-content")]'
                        ).find_elements(By.TAG_NAME, "a")) > 0
                    )

                    group_name = group.find_element(By.TAG_NAME, "p").text.strip()
                    majors = group.find_element(
                        By.XPATH, './/div[contains(@class, "ant-collapse-content")]'
                    ).find_elements(By.TAG_NAME, "a")

                    if group_name not in printed_group_names:
                        print(f"[DEBUG] → Nhóm ngành: {group_name} có {len(majors)} ngành")
                        printed_group_names.add(group_name)

                    for major in majors:
                        major_name = major.text.strip()
                        major_link = major.get_attribute("href")
                        if major_link not in seen_major_links:
                            seen_major_links.add(major_link)
                            group_data.append((group_name, major_name, major_link))

                    success = True
                    break
                except Exception:
                    print(f"[WARN] Thử lần {attempt+1} thất bại khi mở nhóm ngành index {group_index}")
                    time.sleep(1)

            if not success:
                print(f"[ERROR] ❌ Không thể mở nhóm ngành tại index {group_index} sau 3 lần thử.")
        except Exception as e:
            print(f"[WARN] Không thể xử lý nhóm ngành tại index {group_index}")
            print(traceback.format_exc())

except Exception as e:
    print("[ERROR] ❌ Lỗi khi lấy danh sách nhóm ngành/ngành:", e)
    print(traceback.format_exc())

# ========== 4. TIẾN HÀNH CRAWL TỪ DANH SÁCH ==========
data = []
school_counter = 0

for idx, (group_name, major_name, major_link) in enumerate(group_data, 1):
    try:
        print(f"[DEBUG] ({idx}/{len(group_data)}) Đang xử lý ngành: {major_name}")

        for try_load in range(2):
            try:
                driver.set_page_load_timeout(30)
                driver.get(major_link)
                close_ads_if_present(driver, wait)
                break
            except Exception as e:
                print(f"[WARN] ⚠️ Retry load ngành {major_name} lần {try_load+1}: {e}")
                if try_load == 1:
                    raise e

        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ant-table-container")))

        previous_count = 0
        while True:
            school_rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'ant-table-container')]//tbody/tr")
            current_count = len(school_rows)
            if current_count > previous_count:
                previous_count = current_count
                try:
                    see_more_button = WebDriverWait(driver, 3).until(
                        EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'viewmore')]"))
                    )
                    driver.execute_script("arguments[0].click();", see_more_button)
                    print(f"[DEBUG] Đã click 'Xem thêm' → tổng trường hiện tại: {current_count}")
                    time.sleep(2)
                except:
                    print("[DEBUG] Không còn nút 'Xem thêm'.")
                    break
            else:
                break

        school_info = []
        school_rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'ant-table-container')]//tbody/tr")
        for row in school_rows:
            try:
                cols = row.find_elements(By.TAG_NAME, "td")
                if len(cols) >= 5:
                    name = cols[1].text.strip()
                    link = cols[4].find_element(By.TAG_NAME, "a").get_attribute("href")
                    school_info.append((name, link))
            except:
                continue

        for i, (school_name, school_link) in enumerate(school_info, 1):
            print(f"[DEBUG]      ({i}) Trường: {school_name}")
            print(f"[DEBUG]        → Mở link trường: {school_link}")

            for retry in range(2):
                try:
                    driver.set_page_load_timeout(30)
                    driver.get(school_link)
                    close_ads_if_present(driver, wait)
                    wait.until(EC.presence_of_element_located((By.ID, "diem-thi-thpt")))
                    table = driver.find_element(By.ID, "diem-thi-thpt")
                    rows = table.find_elements(By.XPATH, ".//tbody/tr")

                    print(f"[DEBUG]         Số dòng điểm tìm thấy: {len(rows)}")

                    for row in rows:
                        cols = row.find_elements(By.TAG_NAME, "td")
                        if len(cols) >= 6:
                            data.append({
                                "Nhóm ngành": group_name,
                                "Ngành": major_name,
                                "Trường đào tạo": school_name,
                                "Mã ngành": cols[1].text.strip(),
                                "Tên ngành": cols[2].text.strip(),
                                "Tổ hợp môn": cols[3].text.strip(),
                                "Điểm chuẩn 2024": cols[4].text.strip(),
                                "Ghi chú": cols[5].text.strip()
                            })
                            school_counter += 1

                    if school_counter % 10 == 0:
                        print(f"[INFO] Đã lấy được {school_counter} dòng dữ liệu...")
                    break
                except Exception:
                    if retry == 1:
                        print(f"[WARN] ⚠️ Không thể lấy bảng điểm tại trường: {school_name}")
                        print(traceback.format_exc())
                    else:
                        print(f"[WARN] ⚠️ Thử lại lần 2...")

    except Exception:
        print(f"[ERROR] ❌ Lỗi khi xử lý ngành: {major_name}")
        print(traceback.format_exc())

# ========== 5. LƯU FILE ==========
driver.quit()
df = pd.DataFrame(data)
if not df.empty:
    df.drop_duplicates(subset=["Mã ngành", "Trường đào tạo"], keep="first", inplace=True)
    os.makedirs("Data", exist_ok=True)
    df.to_csv("Data/nganh_dao_tao_4.csv", index=False, encoding="utf-8-sig")
    print(f"✅ Đã crawl xong {school_counter} dòng dữ liệu. File đã lưu tại: Data/nganh_dao_tao_4.csv")
else:
    print("⚠️ Không có dữ liệu nào được lưu.")

C:\Users\WIN 10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\undetected_chromedriver\__init__.py:339: UserWarning: using ChromeOptions.user_data_dir might stop working in future versions.use uc.Chrome(user_data_dir='/xyz/some/data') in case you need existing profile folder
  warnings.warn(


[WARN] Không tìm thấy hoặc không thể click nút 'Xem thêm'.
[DEBUG] Đang chờ lấy danh sách nhóm ngành...
[DEBUG] Tìm thấy 34 nhóm ngành.
[DEBUG] → Nhóm ngành: Kế toán - Kiểm toán có 2 ngành
[DEBUG] → Nhóm ngành: Tài chính - Ngân hàng - Bảo hiểm có 7 ngành
[DEBUG] → Nhóm ngành: Kinh tế - Quản trị kinh doanh - Thương Mại có 18 ngành
[DEBUG] → Nhóm ngành: Công nghệ thông tin - Tin học có 23 ngành
[DEBUG] → Nhóm ngành: Báo chí - Marketing - Quảng cáo - PR có 11 ngành
[DEBUG] → Nhóm ngành: Sư phạm - Giáo dục có 35 ngành
[DEBUG] → Nhóm ngành: Y - Dược có 20 ngành
[DEBUG] → Nhóm ngành: Bác sĩ thú y có 3 ngành
[DEBUG] → Nhóm ngành: Công an - Quân đội có 30 ngành
[DEBUG] → Nhóm ngành: Thiết kế đồ họa - Game - Đa phương tiện có 5 ngành
[DEBUG] → Nhóm ngành: Xây dựng - Kiến trúc - Giao thông có 22 ngành
[DEBUG] → Nhóm ngành: Ngoại giao - Ngoại ngữ có 24 ngành
[DEBUG] → Nhóm ngành: Ngoại thương - Xuất nhập khẩu - Kinh Tế quốc tế có 9 ngành
[DEBUG] → Nhóm ngành: Du lịch-Khách sạn có 12 ngành
[DEBUG]

## Đang check xem có oke không thì chrome out of memory xong crash luôn không chạy nữa :(

In [8]:
# ========== 1. SETUP TRÌNH DUYỆT ==========
options = uc.ChromeOptions()
options.add_argument('--disable-gpu')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

profile_directory = "Profile"
os.makedirs(profile_directory, exist_ok=True)
options.user_data_dir = profile_directory

driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, 15)
action = ActionChains(driver)

target_url = "https://diemthi.tuyensinh247.com/nganh-dao-tao.html#undefined"
driver.get(target_url)
close_ads_if_present(driver, wait)

try:
    see_more = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Xem thêm')]"))
    )
    driver.execute_script("arguments[0].click();", see_more)
    print("[DEBUG] Đã click 'Xem thêm' để hiển thị thêm nhóm ngành.")
    time.sleep(2)
except:
    print("[WARN] Không tìm thấy hoặc không thể click nút 'Xem thêm'.")

group_data = []
printed_group_names = set()
seen_major_links = set()

try:
    print("[DEBUG] Đang chờ lấy danh sách nhóm ngành...")
    wait.until(lambda d: len(d.find_elements(By.XPATH, '//div[@role="tablist"]/div')) >= 30)
    groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
    print(f"[DEBUG] Tìm thấy {len(groups)} nhóm ngành.")

    for group_index in range(len(groups)):
        try:
            groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
            group = groups[group_index]
            driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", group)
            time.sleep(1)

            success = False
            for attempt in range(3):
                try:
                    groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
                    group = groups[group_index]
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", group)
                    time.sleep(1.2)

                    try:
                        action.move_to_element(group).click().perform()
                    except:
                        driver.execute_script("arguments[0].click();", group)

                    WebDriverWait(driver, 5).until(
                        lambda d: 'aria-expanded="true"' in group.get_attribute("outerHTML")
                    )

                    WebDriverWait(driver, 5).until(
                        lambda d: len(group.find_element(
                            By.XPATH, './/div[contains(@class, "ant-collapse-content")]'
                        ).find_elements(By.TAG_NAME, "a")) > 0
                    )

                    group_name = group.find_element(By.TAG_NAME, "p").text.strip()
                    majors = group.find_element(
                        By.XPATH, './/div[contains(@class, "ant-collapse-content")]'
                    ).find_elements(By.TAG_NAME, "a")

                    if group_name not in printed_group_names:
                        print(f"[DEBUG] → Nhóm ngành: {group_name} có {len(majors)} ngành")
                        printed_group_names.add(group_name)

                    for major in majors:
                        major_name = major.text.strip()
                        major_link = major.get_attribute("href")
                        if major_link not in seen_major_links:
                            seen_major_links.add(major_link)
                            group_data.append((group_name, major_name, major_link))

                    success = True
                    break
                except Exception:
                    print(f"[WARN] Thử lần {attempt+1} thất bại khi mở nhóm ngành index {group_index}")
                    time.sleep(1)

            if not success:
                print(f"[ERROR] ❌ Không thể mở nhóm ngành tại index {group_index} sau 3 lần thử.")
        except Exception as e:
            print(f"[WARN] Không thể xử lý nhóm ngành tại index {group_index}")
            print(traceback.format_exc())

except Exception as e:
    print("[ERROR] ❌ Lỗi khi lấy danh sách nhóm ngành/ngành:", e)
    print(traceback.format_exc())

data = []
school_counter = 0
crawled_major_names = set()

for idx, (group_name, major_name, major_link) in enumerate(group_data, 1):
    if major_name in crawled_major_names:
        continue  # ✅ bỏ qua ngành đã crawl trước đó

    crawled_major_names.add(major_name)
    print(f"[DEBUG] ({idx}/{len(group_data)}) Đang xử lý ngành: {major_name}")

    # Thử tải trang major 2 lần nếu bị lỗi
    for try_load in range(2):
        try:
            driver.set_page_load_timeout(30)
            driver.get(major_link)
            close_ads_if_present(driver, wait)
            break
        except Exception as e:
            print(f"[WARN] ⚠️ Retry load ngành {major_name} lần {try_load+1}: {e}")
            if try_load == 1:
                continue

    try:
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ant-table-container")))
    except:
        print(f"[WARN] Không thấy bảng trường trong ngành {major_name}, bỏ qua.")
        continue

    # Click 'Xem thêm' nhiều lần
    previous_count = 0
    while True:
        school_rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'ant-table-container')]//tbody/tr")
        current_count = len(school_rows)
        if current_count > previous_count:
            previous_count = current_count
            try:
                see_more_button = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'viewmore')]"))
                )
                driver.execute_script("arguments[0].click();", see_more_button)
                print(f"[DEBUG] Đã click 'Xem thêm' → tổng trường hiện tại: {current_count}")
                time.sleep(2)
            except:
                print("[DEBUG] Không còn nút 'Xem thêm'.")
                break
        else:
            break

    # Lấy danh sách trường
    school_info = []
    school_rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'ant-table-container')]//tbody/tr")
    for row in school_rows:
        try:
            cols = row.find_elements(By.TAG_NAME, "td")
            if len(cols) >= 5:
                name = cols[1].text.strip()
                link = cols[4].find_element(By.TAG_NAME, "a").get_attribute("href")
                school_info.append((name, link))
        except:
            continue

    # Truy cập từng trường
    for i, (school_name, school_link) in enumerate(school_info, 1):
        print(f"[DEBUG]      ({i}) Trường: {school_name}")
        print(f"[DEBUG]        → Mở link trường: {school_link}")

        for retry in range(2):
            try:
                driver.set_page_load_timeout(30)
                driver.get(school_link)
                close_ads_if_present(driver, wait)
                wait.until(EC.presence_of_element_located((By.ID, "diem-thi-thpt")))

                table = driver.find_element(By.ID, "diem-thi-thpt")
                rows = table.find_elements(By.XPATH, ".//tbody/tr")

                print(f"[DEBUG]         Số dòng điểm tìm thấy: {len(rows)}")

                for row in rows:
                    cols = row.find_elements(By.TAG_NAME, "td")
                    if len(cols) >= 6:
                        data.append({
                            "Nhóm ngành": group_name,
                            "Ngành": major_name,
                            "Trường đào tạo": school_name,
                            "Mã ngành": cols[1].text.strip(),
                            "Tên ngành": cols[2].text.strip(),
                            "Tổ hợp môn": cols[3].text.strip(),
                            "Điểm chuẩn 2024": cols[4].text.strip(),
                            "Ghi chú": cols[5].text.strip()
                        })
                        school_counter += 1

                if school_counter % 10 == 0:
                    print(f"[INFO] Đã lấy được {school_counter} dòng dữ liệu...")
                break

            except Exception:
                if retry == 1:
                    print(f"[WARN] ⚠️ Không thể lấy bảng điểm tại trường: {school_name}")
                    print(traceback.format_exc())
                else:
                    print(f"[WARN] ⚠️ Thử lại lần 2...")

# ========== 5. LƯU FILE ==========
driver.quit()
df = pd.DataFrame(data)
if not df.empty:
    df.drop_duplicates(subset=["Mã ngành", "Trường đào tạo"], keep="first", inplace=True)
    os.makedirs("Data", exist_ok=True)
    df.to_csv("Data/nganh_dao_tao_final.csv", index=False, encoding="utf-8-sig")
    print(f"✅ Đã crawl xong {school_counter} dòng dữ liệu. File đã lưu tại: Data/nganh_dao_tao_final.csv")
else:
    print("⚠️ Không có dữ liệu nào được lưu.")

[WARN] Không tìm thấy hoặc không thể click nút 'Xem thêm'.
[DEBUG] Đang chờ lấy danh sách nhóm ngành...
[DEBUG] Tìm thấy 34 nhóm ngành.
[DEBUG] → Nhóm ngành: Kế toán - Kiểm toán có 2 ngành
[DEBUG] → Nhóm ngành: Tài chính - Ngân hàng - Bảo hiểm có 7 ngành
[DEBUG] → Nhóm ngành: Kinh tế - Quản trị kinh doanh - Thương Mại có 18 ngành
[DEBUG] → Nhóm ngành: Công nghệ thông tin - Tin học có 23 ngành
[DEBUG] → Nhóm ngành: Báo chí - Marketing - Quảng cáo - PR có 11 ngành
[DEBUG] → Nhóm ngành: Sư phạm - Giáo dục có 35 ngành
[DEBUG] → Nhóm ngành: Y - Dược có 20 ngành
[DEBUG] → Nhóm ngành: Bác sĩ thú y có 3 ngành
[DEBUG] → Nhóm ngành: Công an - Quân đội có 30 ngành
[DEBUG] → Nhóm ngành: Thiết kế đồ họa - Game - Đa phương tiện có 5 ngành
[DEBUG] → Nhóm ngành: Xây dựng - Kiến trúc - Giao thông có 22 ngành
[DEBUG] → Nhóm ngành: Ngoại giao - Ngoại ngữ có 24 ngành
[DEBUG] → Nhóm ngành: Ngoại thương - Xuất nhập khẩu - Kinh Tế quốc tế có 9 ngành
[DEBUG] → Nhóm ngành: Du lịch-Khách sạn có 12 ngành
[DEBUG]

TimeoutException: Message: timeout: Timed out receiving message from renderer: 30.000
  (Session info: chrome=136.0.7103.49)
Stacktrace:
	GetHandleVerifier [0x0047FBD3+61635]
	GetHandleVerifier [0x0047FC14+61700]
	(No symbol) [0x002A05D3]
	(No symbol) [0x0029116B]
	(No symbol) [0x00290EA1]
	(No symbol) [0x0028EE80]
	(No symbol) [0x0028F93D]
	(No symbol) [0x0029C049]
	(No symbol) [0x002AD6C5]
	(No symbol) [0x002B30E6]
	(No symbol) [0x0028FF4D]
	(No symbol) [0x002AD461]
	(No symbol) [0x0032E52A]
	(No symbol) [0x0030D096]
	(No symbol) [0x002DC840]
	(No symbol) [0x002DD6A4]
	GetHandleVerifier [0x006C7043+2450739]
	GetHandleVerifier [0x006C27C6+2432182]
	GetHandleVerifier [0x006DD50E+2542078]
	GetHandleVerifier [0x00496895+155013]
	GetHandleVerifier [0x0049CF7D+181357]
	GetHandleVerifier [0x00487428+92440]
	GetHandleVerifier [0x004875D0+92864]
	GetHandleVerifier [0x00471FC0+5296]
	BaseThreadInitThunk [0x754EFCC9+25]
	RtlGetAppContainerNamedObjectPath [0x77377C5E+286]
	RtlGetAppContainerName

In [ ]:
data

[{'Nhóm ngành': 'Kế toán - Kiểm toán',
  'Ngành': 'Kế toán',
  'Trường đào tạo': 'Đại Học Kinh Tế Quốc Dân',
  'Mã ngành': '7340301',
  'Tên ngành': 'Kế toán',
  'Tổ hợp môn': 'A00; A01; D01; D07',
  'Điểm chuẩn 2024': '27.29',
  'Ghi chú': ''},
 {'Nhóm ngành': 'Kế toán - Kiểm toán',
  'Ngành': 'Kế toán',
  'Trường đào tạo': 'Đại Học Kinh Tế Quốc Dân',
  'Mã ngành': 'EP04',
  'Tên ngành': 'Kế toán tích hợp chứng chi quốc tế (JCAEW CFAB)',
  'Tổ hợp môn': 'A00; A01; D01; D07',
  'Điểm chuẩn 2024': '27.2',
  'Ghi chú': ''},
 {'Nhóm ngành': 'Kế toán - Kiểm toán',
  'Ngành': 'Kế toán',
  'Trường đào tạo': 'Đại Học Bách Khoa Hà Nội',
  'Mã ngành': 'EM4',
  'Tên ngành': 'Kế toán',
  'Tổ hợp môn': 'A00; A01; D01; K01 (Toán, Văn, Lý/Hóa/Sinh/Tin)',
  'Điểm chuẩn 2024': '25.8',
  'Ghi chú': ''},
 {'Nhóm ngành': 'Kế toán - Kiểm toán',
  'Ngành': 'Kế toán',
  'Trường đào tạo': 'Học Viện Tài chính',
  'Mã ngành': '7340301',
  'Tên ngành': 'Kế toán',
  'Tổ hợp môn': 'A00; A01; D01; D07',
  'Điểm ch

In [10]:
df = pd.DataFrame(data)

df.to_csv("Data/nganh_dao_tao_backup.csv", index=False, encoding="utf-8-sig")
print("✅ Đã lưu backup trước khi tiếp tục.")

✅ Đã lưu backup trước khi tiếp tục.


## Đang cố gắng crawl đoạn tiếp theo sau khi crash

In [5]:
# ========== HÀM TẠO MỚI DRIVER ==========
def create_driver():
    options = uc.ChromeOptions()
    options.add_argument('--disable-gpu')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    profile_directory = "Profile"
    os.makedirs(profile_directory, exist_ok=True)
    options.user_data_dir = profile_directory
    return uc.Chrome(options=options)

# ========== HÀM RELOAD DRIVER ==========
def reload_driver():
    global driver, wait, action
    try:
        driver.quit()
    except:
        pass
    time.sleep(3)
    print("[RELOAD] Đang reload lại trình duyệt...")
    driver = create_driver()
    wait = WebDriverWait(driver, 15)
    action = ActionChains(driver)

# ========== KHỞI TẠO ==========
reload_driver()

# ========== LOAD BACKUP ==========
backup_file = "Data/nganh_dao_tao_backup.csv"
final_file = "Data/nganh_dao_tao_final.csv"
data = []

if os.path.exists(backup_file):
    df_backup = pd.read_csv(backup_file)
    data = df_backup.to_dict("records")
    print(f"[RESUME] Đã đọc {len(data)} dòng từ file backup.")

crawled_major_names = set(d["Ngành"] for d in data)

# ========== TRUY CẬP TRANG ==========
driver.get("https://diemthi.tuyensinh247.com/nganh-dao-tao.html#undefined")
close_ads_if_present(driver, wait)

# Click nút "Xem thêm" nếu có
try:
    see_more = WebDriverWait(driver, 5).until(
        EC.element_to_be_clickable((By.XPATH, "//button[contains(text(),'Xem thêm')]"))
    )
    driver.execute_script("arguments[0].click();", see_more)
    print("[DEBUG] Đã click 'Xem thêm' để hiển thị thêm nhóm ngành.")
    time.sleep(2)
except:
    print("[WARN] Không tìm thấy nút 'Xem thêm'.")

# ========== LẤY DANH SÁCH NHÓM NGÀNH + NGÀNH ==========
group_data = []
printed_group_names = set()
seen_major_links = set()

try:
    print("[DEBUG] Đang chờ lấy danh sách nhóm ngành...")
    wait.until(lambda d: len(d.find_elements(By.XPATH, '//div[@role="tablist"]/div')) >= 30)
    groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
    print(f"[DEBUG] Tìm thấy {len(groups)} nhóm ngành.")
    for group_index in range(len(groups)):
        for attempt in range(3):
            try:
                groups = driver.find_elements(By.XPATH, '//div[@role="tablist"]/div')
                group = groups[group_index]
                group_name = group.find_element(By.TAG_NAME, "p").text.strip()
                driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", group)
                time.sleep(1.2)
                try:
                    action.move_to_element(group).click().perform()
                except:
                    driver.execute_script("arguments[0].click();", group)

                WebDriverWait(driver, 5).until(
                    lambda d: 'aria-expanded="true"' in group.get_attribute("outerHTML")
                )
                majors = group.find_element(
                    By.XPATH, './/div[contains(@class, "ant-collapse-content")]'
                ).find_elements(By.TAG_NAME, "a")

                if group_name not in printed_group_names:
                    print(f"[DEBUG] → Nhóm ngành: {group_name} có {len(majors)} ngành")
                    printed_group_names.add(group_name)

                for major in majors:
                    major_name = major.text.strip()
                    major_link = major.get_attribute("href")
                    if major_link not in seen_major_links:
                        seen_major_links.add(major_link)
                        group_data.append((group_name, major_name, major_link))
                break
            except:
                print(f"[WARN] Lỗi mở nhóm ngành {group_index}, thử lại ({attempt+1}/3)")
except Exception as e:
    print("[ERROR] ❌ Lỗi khi lấy danh sách ngành:", e)
    print(traceback.format_exc())

# ========== CRAWL DỮ LIỆU ==========
school_counter = len(data)
for idx, (group_name, major_name, major_link) in enumerate(group_data, 1):
    if major_name in crawled_major_names:
        continue

    print(f"[DEBUG] ({idx}/{len(group_data)}) Đang xử lý ngành: {major_name}")
    for try_load in range(2):
        try:
            driver.set_page_load_timeout(30)
            driver.get(major_link)
            close_ads_if_present(driver, wait)
            break
        except:
            print(f"[RETRY] Thử lại ngành {major_name}")
            if try_load == 1:
                continue

    try:
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, "ant-table-container")))
    except:
        print(f"[WARN] Không thấy bảng trường trong ngành {major_name}, bỏ qua.")
        continue

    # Click "Xem thêm"
    previous_count = 0
    while True:
        school_rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'ant-table-container')]//tbody/tr")
        current_count = len(school_rows)
        if current_count > previous_count:
            previous_count = current_count
            try:
                see_more_button = WebDriverWait(driver, 3).until(
                    EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'viewmore')]"))
                )
                driver.execute_script("arguments[0].click();", see_more_button)
                print(f"[DEBUG] Click 'Xem thêm' → {current_count}")
                time.sleep(2)
            except:
                print("[DEBUG] Không còn nút 'Xem thêm'.")
                break
        else:
            break

    school_info = []
    school_rows = driver.find_elements(By.XPATH, "//div[contains(@class, 'ant-table-container')]//tbody/tr")
    for row in school_rows:
        try:
            cols = row.find_elements(By.TAG_NAME, "td")
            if len(cols) >= 5:
                name = cols[1].text.strip()
                link = cols[4].find_element(By.TAG_NAME, "a").get_attribute("href")
                school_info.append((name, link))
        except:
            continue

    for i, (school_name, school_link) in enumerate(school_info, 1):
        print(f"[DEBUG]      ({i}) Trường: {school_name}")
        print(f"[DEBUG]        → Link: {school_link}")
        for retry in range(2):
            try:
                driver.set_page_load_timeout(30)
                driver.get(school_link)
                close_ads_if_present(driver, wait)
                wait.until(EC.presence_of_element_located((By.ID, "diem-thi-thpt")))
                table = driver.find_element(By.ID, "diem-thi-thpt")
                rows = table.find_elements(By.XPATH, ".//tbody/tr")

                print(f"[DEBUG]         Số dòng điểm tìm thấy: {len(rows)}")
                for row in rows:
                    cols = row.find_elements(By.TAG_NAME, "td")
                    if len(cols) >= 6:
                        data.append({
                            "Nhóm ngành": group_name,
                            "Ngành": major_name,
                            "Trường đào tạo": school_name,
                            "Mã ngành": cols[1].text.strip(),
                            "Tên ngành": cols[2].text.strip(),
                            "Tổ hợp môn": cols[3].text.strip(),
                            "Điểm chuẩn 2024": cols[4].text.strip(),
                            "Ghi chú": cols[5].text.strip()
                        })
                        school_counter += 1
                if school_counter % 10 == 0:
                    print(f"[INFO] Đã lấy được {school_counter} dòng dữ liệu...")
                break
            except:
                print("[RETRY] Thử lại trường...")
                reload_driver()
                break
    crawled_major_names.add(major_name)

    # Lưu backup mỗi vòng
    pd.DataFrame(data).to_csv(backup_file, index=False, encoding="utf-8-sig")

# ===== Lưu file kết quả =====
driver.quit()
df = pd.DataFrame(data)
if not df.empty:
    df.drop_duplicates(subset=["Mã ngành", "Trường đào tạo"], inplace=True)
    os.makedirs("Data", exist_ok=True)
    df.to_csv(final_file, index=False, encoding="utf-8-sig")
    print(f"✅ Đã crawl xong {school_counter} dòng dữ liệu. File đã lưu tại: {final_file}")
else:
    print("⚠️ Không có dữ liệu nào được lưu.")

[RELOAD] Đang reload lại trình duyệt...


C:\Users\WIN 10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\undetected_chromedriver\__init__.py:339: UserWarning: using ChromeOptions.user_data_dir might stop working in future versions.use uc.Chrome(user_data_dir='/xyz/some/data') in case you need existing profile folder
  warnings.warn(


[RESUME] Đã đọc 3909 dòng từ file backup.
[WARN] Không tìm thấy nút 'Xem thêm'.
[DEBUG] Đang chờ lấy danh sách nhóm ngành...
[DEBUG] Tìm thấy 34 nhóm ngành.
[DEBUG] → Nhóm ngành: Kế toán - Kiểm toán có 2 ngành
[DEBUG] → Nhóm ngành: Tài chính - Ngân hàng - Bảo hiểm có 7 ngành
[DEBUG] → Nhóm ngành: Kinh tế - Quản trị kinh doanh - Thương Mại có 18 ngành
[DEBUG] → Nhóm ngành: Công nghệ thông tin - Tin học có 23 ngành
[DEBUG] → Nhóm ngành: Báo chí - Marketing - Quảng cáo - PR có 11 ngành
[WARN] Lỗi mở nhóm ngành 5, thử lại (1/3)
[DEBUG] → Nhóm ngành: Sư phạm - Giáo dục có 35 ngành
[WARN] Lỗi mở nhóm ngành 6, thử lại (1/3)
[DEBUG] → Nhóm ngành: Y - Dược có 20 ngành
[DEBUG] → Nhóm ngành: Bác sĩ thú y có 3 ngành
[DEBUG] → Nhóm ngành: Công an - Quân đội có 30 ngành
[DEBUG] → Nhóm ngành: Thiết kế đồ họa - Game - Đa phương tiện có 5 ngành
[DEBUG] → Nhóm ngành: Xây dựng - Kiến trúc - Giao thông có 22 ngành
[DEBUG] → Nhóm ngành: Ngoại giao - Ngoại ngữ có 24 ngành
[DEBUG] → Nhóm ngành: Ngoại thương 

C:\Users\WIN 10\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\undetected_chromedriver\__init__.py:339: UserWarning: using ChromeOptions.user_data_dir might stop working in future versions.use uc.Chrome(user_data_dir='/xyz/some/data') in case you need existing profile folder
  warnings.warn(


[DEBUG] (47/401) Đang xử lý ngành: Thiết kế vi mạch
[DEBUG] Không còn nút 'Xem thêm'.
[DEBUG]      (1) Trường: Trường Đại Học Bách Khoa HCM
[DEBUG]        → Link: https://diemthi.tuyensinh247.com/nganh-dao-tao/thiet-ke-vi-mach-47.html?truong=QSB
[RETRY] Thử lại trường...
[RELOAD] Đang reload lại trình duyệt...
[DEBUG] (60/401) Đang xử lý ngành: 
[DEBUG] Không còn nút 'Xem thêm'.
[DEBUG]      (1) Trường: Đại Học Bách Khoa Hà Nội
[DEBUG]        → Link: https://diemthi.tuyensinh247.com/nganh-dao-tao/cong-nghe-giao-duc-60.html?truong=BKA
[DEBUG]         Số dòng điểm tìm thấy: 1
[INFO] Đã lấy được 3910 dòng dữ liệu...
[DEBUG]      (2) Trường: Trường Đại Học Khoa Học Tự Nhiên TPHCM
[DEBUG]        → Link: https://diemthi.tuyensinh247.com/nganh-dao-tao/cong-nghe-giao-duc-60.html?truong=QST
[DEBUG]         Số dòng điểm tìm thấy: 1
[DEBUG]      (3) Trường: Trường Đại Học Sư Phạm Kỹ Thuật Hưng Yên
[DEBUG]        → Link: https://diemthi.tuyensinh247.com/nganh-dao-tao/cong-nghe-giao-duc-60.html?tru

In [14]:
# merge Data/nganh_dao_tao_backup.csv và Data/nganh_dao_tao_final.csv
df_backup = pd.read_csv(backup_file)
df_final = pd.read_csv(final_file)
df_combined = pd.concat([df_backup, df_final], ignore_index=True)
df_combined

,Nhóm ngành,Ngành,Trường đào tạo,Mã ngành,Tên ngành,Tổ hợp môn,Điểm chuẩn 2024,Ghi chú
0,Kế toán - Kiểm toán,Kế toán,Đại Học Kinh Tế Quốc Dân,7340301,Kế toán,A00; A01; D01; D07,27.29,NaN
1,Kế toán - Kiểm toán,Kế toán,Đại Học Kinh Tế Quốc Dân,EP04,Kế toán tích hợp chứng chi quốc tế (JCAEW CFAB),A00; A01; D01; D07,27.20,NaN
2,Kế toán - Kiểm toán,Kế toán,Đại Học Bách Khoa Hà Nội,EM4,Kế toán,"A00; A01; D01; K01 (Toán, Văn, Lý/Hóa/Sinh/Tin)",25.80,NaN
3,Kế toán - Kiểm toán,Kế toán,Học Viện Tài chính,7340301,Kế toán,A00; A01; D01; D07,26.45,NaN
4,Kế toán - Kiểm toán,Kế toán,Học Viện Tài chính,7340301C21,Kế toán doanh nghiệp (theo định hướng ACCA),A01; D01; D07,34.35,"CT định hướng CCQT, Tiếng Anh hệ số 2"
...,...,...,...,...,...,...,...,...
11252,Khoa học tư nhiên khác,Vật lý học,Trường Đại Học Sư Phạm TPHCM,7440102,Vật lý học,A00; A01; D90,24.44,NaN
11253,Khoa học tư nhiên khác,Vật lý học,Trường Đại Học Khoa Học Tự Nhiên TPHCM,7440102_DKD,Vật lý học (CT tăng cường tiếng Anh),"A00; A01; C01; C03; A04; A10; (Toán, Lí, Công ...",NaN,NaN
11254,Khoa học tư nhiên khác,Vật lý học,Trường Đại Học Khoa Học Tự Nhiên Hà Nội,QHT03,Vật lý học,A00; A01; A02; C01,25.30,NaN
11255,Khoa học tư nhiên khác,Vật lý học,Trường Đại Học Khoa Học Huế,7440102,Vật lý học,"A00; A01; C01; (Toán, Lí, Tin)",NaN,NaN


In [15]:
# drop duplicates rows
df_combined.drop_duplicates(keep="first", inplace=True)

In [16]:
df_combined

,Nhóm ngành,Ngành,Trường đào tạo,Mã ngành,Tên ngành,Tổ hợp môn,Điểm chuẩn 2024,Ghi chú
0,Kế toán - Kiểm toán,Kế toán,Đại Học Kinh Tế Quốc Dân,7340301,Kế toán,A00; A01; D01; D07,27.29,NaN
1,Kế toán - Kiểm toán,Kế toán,Đại Học Kinh Tế Quốc Dân,EP04,Kế toán tích hợp chứng chi quốc tế (JCAEW CFAB),A00; A01; D01; D07,27.20,NaN
2,Kế toán - Kiểm toán,Kế toán,Đại Học Bách Khoa Hà Nội,EM4,Kế toán,"A00; A01; D01; K01 (Toán, Văn, Lý/Hóa/Sinh/Tin)",25.80,NaN
3,Kế toán - Kiểm toán,Kế toán,Học Viện Tài chính,7340301,Kế toán,A00; A01; D01; D07,26.45,NaN
4,Kế toán - Kiểm toán,Kế toán,Học Viện Tài chính,7340301C21,Kế toán doanh nghiệp (theo định hướng ACCA),A01; D01; D07,34.35,"CT định hướng CCQT, Tiếng Anh hệ số 2"
...,...,...,...,...,...,...,...,...
6189,Khoa học tư nhiên khác,Vật lý học,Trường Đại Học Khoa Học Tự Nhiên TPHCM,7440102_DKD,Vật lý học (CT tăng cường tiếng Anh),"A00; A01; C01; C03; A04; A10; (Toán, Lí, Công ...",NaN,NaN
6190,Khoa học tư nhiên khác,Vật lý học,Trường Đại Học Khoa Học Tự Nhiên TPHCM,7440102_NN,"Nhóm ngành Vật lý học, Công nghệ Vật lý điện t...","A00; A01; C01; C03; A04; A10; (Toán, Lí, Công ...",NaN,NaN
6191,Khoa học tư nhiên khác,Vật lý học,Trường Đại Học Khoa Học Tự Nhiên Hà Nội,QHT03,Vật lý học,A00; A01; A02; C01,25.30,NaN
6192,Khoa học tư nhiên khác,Vật lý học,Trường Đại Học Khoa Học Huế,7440102,Vật lý học,"A00; A01; C01; (Toán, Lí, Tin)",NaN,NaN


In [17]:
df_combined.drop_duplicates(keep="first", inplace=True)

In [18]:
# drop cột có điểm chuẩn là NaN
df_combined.dropna(subset=["Điểm chuẩn 2024"], inplace=True)
df_combined.reset_index(drop=True, inplace=True)
df_combined.to_csv("Data/diem_chuan_2024_check_var.csv", index=False, encoding="utf-8-sig")